<a href="https://colab.research.google.com/github/bodamohannaik/DLAI-TF-DPC/blob/master/C3/W4/more_lyrics_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf

# Data Preparation

In [2]:
!wget https://raw.githubusercontent.com/https-deeplearning-ai/tensorflow-1-public/main/C3/W4/misc/Laurences_generated_poetry.txt

--2022-05-28 11:36:21--  https://raw.githubusercontent.com/https-deeplearning-ai/tensorflow-1-public/main/C3/W4/misc/Laurences_generated_poetry.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69001 (67K) [text/plain]
Saving to: ‘Laurences_generated_poetry.txt’

Laurences_generated 100%[===================>]  67.38K  --.-KB/s    in 0.001s  

2022-05-28 11:36:21 (61.7 MB/s) - ‘Laurences_generated_poetry.txt’ saved [69001/69001]



In [3]:
with open('Laurences_generated_poetry.txt', 'r') as fp:
  data = fp.read()
data[:100]

'Come all ye maidens young and fair\nAnd you that are blooming in your prime\nAlways beware and keep yo'

In [4]:
sentences = data.split("\n")
sentences[:5]

['Come all ye maidens young and fair',
 'And you that are blooming in your prime',
 'Always beware and keep your garden fair',
 'Let no man steal away your thyme',
 'For thyme it is a precious thing']

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, lower=True, split=' ', oov_token="<OOV>")
# feed
tokenizer.fit_on_texts(texts=sentences)

In [6]:
# encode to word index
sequences =  tokenizer.texts_to_sequences(texts=sentences)

In [7]:
type(sequences), type(sequences[0])

(list, list)

In [8]:
# prepare n length sequence to predict n+1 word
sliced_sequences = []
minimum_words = 1
for sequence in sequences:
  for i in range(minimum_words, len(sequence)):
    sliced_sequences.append(sequence[:i+1])
print(f"sequences len: {len(sequences)}")
print(f"sliced_sequences len: {len(sliced_sequences)}")

sequences len: 1692
sliced_sequences len: 12038


In [9]:
print((tokenizer.sequences_to_texts(sliced_sequences[0:5])))

['come all', 'come all ye', 'come all ye maidens', 'come all ye maidens young', 'come all ye maidens young and']


In [10]:
# pre pad the sequences
p_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=sliced_sequences, maxlen=None, padding='pre')

In [11]:
p_sequences.shape

(12038, 16)

In [12]:
# split  into input and output
X = p_sequences[:, :-1]
y = p_sequences[:, -1]
X.shape, y.shape

((12038, 15), (12038,))

In [13]:
y = tf.keras.utils.to_categorical(y)

In [14]:
vocab_size = len(tokenizer.word_index)+1

In [15]:
vocab_size

2691

In [16]:
len(y[0])

2691

In [17]:
y.shape

(12038, 2691)

# Model

In [18]:
model = tf.keras.Sequential(
    layers = [
              tf.keras.layers.Input(shape=X.shape[1:]),
              tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=32),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64)),
              tf.keras.layers.Dense(units=vocab_size, activation='softmax')
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 32)            86112     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 2691)              347139    
                                                                 
Total params: 482,915
Trainable params: 482,915
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=200)

Epoch 1/200
377/377 [==============================] - 10s 7ms/step - loss: 6.8014 - accuracy: 0.0650
Epoch 2/200
377/377 [==============================] - 3s 8ms/step - loss: 6.3279 - accuracy: 0.0665
Epoch 3/200
377/377 [==============================] - 3s 7ms/step - loss: 6.1703 - accuracy: 0.0765
Epoch 4/200
377/377 [==============================] - 3s 7ms/step - loss: 5.9911 - accuracy: 0.0846
Epoch 5/200
377/377 [==============================] - 3s 7ms/step - loss: 5.8124 - accuracy: 0.0920
Epoch 6/200
377/377 [==============================] - 3s 7ms/step - loss: 5.6445 - accuracy: 0.0978
Epoch 7/200
377/377 [==============================] - 3s 7ms/step - loss: 5.4885 - accuracy: 0.1052
Epoch 8/200
377/377 [==============================] - 3s 7ms/step - loss: 5.3474 - accuracy: 0.1112
Epoch 9/200
377/377 [==============================] - 3s 7ms/step - loss: 5.2107 - accuracy: 0.1177
Epoch 10/200
377/377 [==============================] - 3s 7ms/step - loss: 5.0759 - accur

In [20]:
text = "Always beware and keep your"
predict_words = 20
sequence = tokenizer.texts_to_sequences([text])[0]
print(sequence)
for i in range(20):
  current_sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=X.shape[1], padding="pre", truncating="pre")
  pred = model.predict(current_sequence)
  current_pred_index = pred.argmax(axis=1)[0]
  if current_pred_index:
    sequence.append(current_pred_index)

[273, 799, 3, 205, 25]


In [21]:
print(tokenizer.sequences_to_texts([sequence]))


['always beware and keep your garden fair hair me dead their fathers coast of clare haste to sure playboy friends great light look dublin breast']
